In [1]:
import requests

postcode_data_url = "http://www.nomisweb.co.uk/output/census/2011/Postcode_Estimates_Table_1.csv"

with requests.Session() as s:
    download = s.get(postcode_data_url)
    decoded_content = download.content.decode('utf-8')

csv_file = open("Postcode_Estimates_Table_1.csv", "w")
csv_file.write(decoded_content)
csv_file.close()



In [5]:
import pandas as pd

df_postcodes = pd.read_csv("Postcode_Estimates_Table_1.csv")
df_postcodes

,Postcode,Total,Males,Females,Occupied_Households
0,AL1 1AG,14,6,8,6
1,AL1 1AJ,124,60,64,51
2,AL1 1AR,32,17,15,17
3,AL1 1AS,34,17,17,13
4,AL1 1BH,52,15,37,41
...,...,...,...,...,...
1308775,YO8 9YA,23,14,9,8
1308776,YO8 9YB,33,17,16,13
1308777,YO8 9YD,9,4,5,4
1308778,YO8 9YE,13,6,7,3


# Data Cleaning

## Postcode Format

UK postcodes can be in inconsistent formats so we should first do some work to check out how they are formatted.

First we want to explore the format of the postcode strings. A good first check is to see if they are all the same length:

In [10]:
#check the lengths of the postcode strings
df_postcodes["Postcode"].apply(len).value_counts()


7    1308780
Name: Postcode, dtype: int64

In [ ]:
They are all of length 7, that is a good start.

We'll use regex with capture groups to grab the different parts of the postcode. Postcodes can be in one of 6 formats:

![UK Postcode Formats](uk_postcode_format.jpg)

Obtained from https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/283357/ILRSpecification2013_14Appendix_C_Dec2012_v1.pdf

Inspecting the postcodes in the data set they are arranged slightly differently:
 - Fixed length of 7 characters.
 - Outcodes left aligned.
 - Incodes right aligned.
 - Spaces in between the outcodes and incodes where they are not 7 characters long.

We will use 6 regex strings and test them on each postcode. Where the regex matches, we will use match groups to extract the 

In [ ]:
postcode_regexes = [
    ""
]